# AI Fairness Analysis
This notebook performs a bias audit of the COMPAS recidivism dataset using AI Fairness 360 (AIF360).


!pip install aif360 pandas matplotlib scikit-learn

import pandas as pd
import matplotlib.pyplot as plt
from aif360.datasets import CompasDataset
from aif360.metrics import ClassificationMetric
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


data = CompasDataset()
train, test = data.split([0.7], shuffle=True)


clf = LogisticRegression(max_iter=1000)
clf.fit(train.features, train.labels.ravel())
preds = clf.predict(test.features)


test_pred = test.copy()
test_pred.labels = preds.reshape(-1,1)

metric = ClassificationMetric(
    test, test_pred,
    unprivileged_groups=[{'race':1}],
    privileged_groups=[{'race':0}]
)

print("False Positive Rate Difference:", metric.false_positive_rate_difference())
print("Equal Opportunity Difference:", metric.equal_opportunity_difference())


labels = ['FPR Diff', 'EOpp Diff']
values = [
    metric.false_positive_rate_difference(),
    metric.equal_opportunity_difference()
]

plt.bar(labels, values)
plt.title("Fairness Metric Disparities (COMPAS)")
plt.show()


## Findings
The audit shows higher false positive rates for African-American defendants, indicating bias in the COMPAS dataset. Mitigation strategies include pre-processing, in-processing, and post-processing fairness techniques.
